In [1]:
import os

os.chdir("/mnt/bn/rui-picocode-lf/OminiControl")
os.environ['HF_HOME'] = "/home/tiger/.cache/huggingface_re20250424"
os.environ['HF_HUB_OFFLINE'] = '1'

print(os.environ["HF_HOME"], os.environ["HF_HUB_OFFLINE"])

/home/tiger/.cache/huggingface_re20250424 1


In [2]:
import torch
from diffusers.pipelines import FluxPipeline
from src.flux.condition import Condition
from PIL import Image

from src.flux.generate import generate, seed_everything

In [3]:
pipe = FluxPipeline.from_pretrained(
    "black-forest-labs/FLUX.1-dev", torch_dtype=torch.bfloat16
)

pipe = pipe.to("cuda")

Couldn't connect to the Hub: Cannot reach https://huggingface.co/api/models/black-forest-labs/FLUX.1-dev: offline mode is enabled. To disable it, please unset the `HF_HUB_OFFLINE` environment variable..
Will try to load from local cache.


OSError: We couldn't connect to 'https://huggingface.co' to load this model, couldn't find it in the cached files and it looks like /home/tiger/.cache/huggingface_re20250424/hub/models--black-forest-labs--FLUX.1-dev/snapshots/0ef5fff789c832c5c7f4e127f94c8b54bbcced44/model_index.json is not the path to a directory containing a model_index.json file.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/diffusers/installation#offline-mode'.

In [ ]:
pipe.load_lora_weights(
    "Yuanshi/OminiControl",
    weight_name=f"experimental/fill.safetensors",
    adapter_name="fill",
)

In [ ]:
image = Image.open("assets/monalisa.jpg").convert("RGB").resize((512, 512))

masked_image = image.copy()
masked_image.paste((0, 0, 0), (128, 100, 384, 220))

condition = Condition("fill", masked_image)

seed_everything()
result_img = generate(
    pipe,
    prompt="The Mona Lisa is wearing a white VR headset with 'Omini' written on it.",
    conditions=[condition],
).images[0]

concat_image = Image.new("RGB", (1536, 512))
concat_image.paste(image, (0, 0))
concat_image.paste(condition.condition, (512, 0))
concat_image.paste(result_img, (1024, 0))
concat_image

In [ ]:
image = Image.open("assets/book.jpg").convert("RGB").resize((512, 512))

w, h, min_dim = image.size + (min(image.size),)
image = image.crop(
    ((w - min_dim) // 2, (h - min_dim) // 2, (w + min_dim) // 2, (h + min_dim) // 2)
).resize((512, 512))


masked_image = image.copy()
masked_image.paste((0, 0, 0), (150, 150, 350, 250))
masked_image.paste((0, 0, 0), (200, 380, 320, 420))

condition = Condition("fill", masked_image)

seed_everything()
result_img = generate(
    pipe,
    prompt="A yellow book with the word 'OMINI' in large font on the cover. The text 'for FLUX' appears at the bottom.",
    conditions=[condition],
).images[0]

concat_image = Image.new("RGB", (1536, 512))
concat_image.paste(image, (0, 0))
concat_image.paste(condition.condition, (512, 0))
concat_image.paste(result_img, (1024, 0))
concat_image